# Purpose

South Korea entered into a trade agreement with the US that was implemented in March 2012.

To examine the influence of the tariff adjustments under the agreement, data pertaining to aluminum powder imports over the years 2008 to 2016, and the tariffs thereof, were the target of the following ETL process. Note that aluminum powder has a tariff duty Ad Valorum, meaning as a percentage of cost.

Text files of the tariff data and .csv files of the imports were extracted from government databases into pandas dataframes, cleaned using the pandas library, and then uploaded to a SQLite table using SQLalchemy in tandem with pandas.

**Information sources**:  

* U.S. - Korea Free Trade Agreement. (n.d.). Retrieved from https://ustr.gov/trade-agreements/free-trade-agreements/korus-fta

**Data sources**:  

_Imports_
* Product Profiles of U.S. Merchandise Trade with a Selected Market. (n.d.). Retrieved from http://tse.export.gov/tse/
    * searched for Selected Markets: "Geographic Regions/Asia", then selected "flow: imports" and "item: HS - 76" for aluminum products, "Display Data for: 2008 to: 2016"  


* Global Patterns of U.S. Merchandise Trade (n.d.). Retrieved from http://tse.export.gov/tse/
    * searched for Selected Markets: Selected "flow: imports" and "item: HS - 7603" for aluminum powder and flakes, "Display Data for: 2008 to: 2016"  


_Tariffs_
* Annual Tariff Data. (n.d.). Retrieved from https://dataweb.usitc.gov/tariff/annual


In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Using SQLite so as not to need a localhost instance, but could easily be adapted to mysql using PyMysql library
engine = create_engine("sqlite://", echo=False)

# Extract

Data was downloaded from http://tse.export.gov/tse/ as .txt files, which are saved in the local resources folder. 

The following extracts the data into dataframes for cleanup. Only the years 2008 to 2016 are to be analyzed

In [3]:
# File route for the a loop to extract data from csvs
file_route = "resources"
years = list(range(2008,2017))
years = [str(year) for year in years]

In [4]:
# Extracting import data from Asia as a whole from .csv file
asia_df = pd.read_csv("resources/DATA_HS_76__ALUMINUM_AND_ARTICLES_THEREOF_Imports_from_Asia.csv")
# put into series
asia_S = asia_df.loc[14, years]

# South Korea is a row in the World data .csv file
korea_from_world=pd.read_csv("resources/DATA_Imports_of_HS_7603__ALUMINUM_POWDERS_AND_FLAKES.csv")
# put into series
sk_S = korea_from_world.loc[7,:]

In [5]:
# Import the csv data to a dataframe for cleanup
data_list = []
for year in years:
    directory = f"{file_route}/{year}.txt"
    
    # the .txt values are separated by the pipe (|) character, 
    # which pd.read_csv can readily handle.
    data = pd.read_csv(directory, sep='|', low_memory=False, dtype={'hts8':str})
    
    # some of the .txt files contain duplicate rows, which are dropped as part of this loop.
    data.drop_duplicates(subset='hts8', inplace=True)
    data_list.append(data)

# Transform

The dataframes formed during extraction contain 111 columns and over 10,000 rows, and the target data regards only aluminum powders and flakes (hts8 codes 7603-1000 and 7603-2000).

In [7]:
# transform imports data

# combining Asia-at-large and South Korea series into DF
sk_comparison = pd.DataFrame([sk_S, asia_S])
# transpose to make years column as index
sk_comparison = sk_comparison.transpose()
# drop unnecessary row
sk_comparison.drop(['Partner'], inplace=True)
# rename columns
sk_comparison.reset_index(inplace=True)
sk_comparison.columns = ['year', 'south_korea','asia']

# show results
sk_comparison.head()

,year,south_korea,asia
0,2008,3268574,5563713
1,2009,1585607,2877272
2,2010,1684673,3743088
3,2011,1988058,4026873
4,2012,2311044,5470198


In [8]:
# transform tariffs data

subdata_list = []

for df, year in zip(data_list,years):
    # select out pertinant columns from overall dataframe for each year
    subdata = df.loc[(df['hts8']=='76031000') | (df['hts8']=='76032000'), ['hts8', 'brief_description', 'mfn_ad_val_rate', 'korea_ad_val_rate']]
    # append year column to the dataframe
    subdata['year'] = [year, year]
    # append dataframe to list for later concatination
    subdata_list.append(subdata)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [10]:
# make final dataframe from list of sub-dataframes
alum_data = pd.concat(subdata_list)

# fill korea_ad_val_rate NaN values to match the general mfn_ad_val_rate 
# for years before trade agreement (2008-2011)
alum_data['korea_ad_val_rate'].fillna(alum_data['mfn_ad_val_rate'], inplace=True)
alum_data.reset_index(inplace=True, drop=True)

In [11]:
# show results
alum_data.head()

,hts8,brief_description,mfn_ad_val_rate,korea_ad_val_rate,year
0,76031000,"Aluminum, powders of non-lamellar structure",0.050,0.050,2008
1,76032000,"Aluminum, powders of lamellar structure; alumi...",0.039,0.039,2008
2,76031000,"Aluminum, powders of non-lamellar structure",0.050,0.050,2009
3,76032000,"Aluminum, powders of lamellar structure; alumi...",0.039,0.039,2009
4,76031000,"Aluminum, powders of non-lamellar structure",0.050,0.050,2010


# Load

Upload aluminum tariff data and South Korea/Asia imports data to SQLite as tables for the analysts. Separate tables are uploaded for tariffs and for imports, otherwise the import data would have to be repeated excessively in a join. Analyst can choose whether they would like to examine powder, flakes, or both via a join in SQL.

In [12]:
alum_data.to_sql('tariffs', con=engine, if_exists='replace')

In [13]:
sk_comparison.to_sql('imports', con=engine, if_exists='replace')